In [ ]:
import sys
sys.dont_write_bytecode = True
sys.path.append('../../lb')
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import cmath
from numba import njit, f8,i8,c16
import numba
import matplotlib.animation as animation
from IPython.display import HTML
import copy
import matplotlib
import random
import  concurrent.futures
import func_for_paralell

import RungeKutta_acceralated

#パラメーター
nu = 0.001

f = (1 + 1j) * 5 * 0.001

Dimention = 14 #uの次元数
q = 2
k_0 = 2 ** (-4)
k_n = np.zeros((Dimention+4,1))
k_n[2:-2,0] = [k_0 * q ** (i+1) for i in range(Dimention)]
#print(k_n)

c_n_1 = copy.deepcopy(k_n)[2:-2, 0:]
c_n_1[-2:, 0:] = np.zeros((2,1))
#print(c_n_1)

beta = 0.5
c_n_2 = copy.deepcopy(k_n)[1:-3, 0:] * -beta
c_n_2[-1, 0] = np.zeros((1, 1))
#print(c_n_2)

c_n_3 = copy.deepcopy(k_n)[:-4, 0:] * (beta - 1)
#print(c_n_3)

#parameter = [nu,f,k_n,c_n_1,c_n_2,c_n_3]
parameter = {
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3
}
#print(parameter)
#print(k_n)

runge_para = {
    'start' :  np.load("14次元初期値.npy"),
    'step' : 0.00025,
    'period' : np.array([0,5000]),
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3
}

# コンテスト(初期値, nu)

# シングルコア

## nu固定

In [ ]:
#ある程度持続すれば更新
runge_para["period"]=np.array([0,1000])
competitives = 100

#刻み幅
runge_para["step"] = 0.01
skip= 100

runge_para['start'] = np.load("../laminar_initials/beta0.5_nu0.0001415923755756859_step0.01_654.27999999955period_laminar.npz")["start"]

#nu
runge_para["nu"] = 0.0001415923755756859
beta= 0.5
#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)
model = RungeKutta_acceralated.RungeKutta()
lmodel = Long_laminar()
for _ in range(3):
    #摂動を与えた初期値のリスト作成
    starts = np.zeros((14,competitives),dtype = np.complex128)
    starts[:,0:1] = runge_para["start"].copy()
    for i in range(competitives-1):
        starts[:,i+1:i+2] = lmodel.perturbator(runge_para['start'])


    #ラミナーから外れた時間
    times = np.zeros((competitives)) + runge_para["period"][1]

    for i in tqdm(range(competitives)):
        runge_para['start'] = starts[:,i:i+1].copy()
        #インスタンス化&実行
        model.get_arr_latter(**runge_para,latter=1)

        #極大抽出
        loc_max = Long_laminar.loc_max_7(model.arr_latter) #u1が極大値なものを拾ってくる
        loc_min = Long_laminar.loc_min_7(model.arr_latter) #u1が極小値なものを拾ってくる

        #極大値で判定
        for j in range(loc_max.shape[1]):
            if not 0.87 < loc_max[0,j] < 1.1:
                times[i] = loc_max[-1, j]
                break
        #極小値で判定
        for j in range(loc_min.shape[1]):
            if not 0.5 < loc_min[0,j] < 0.64:
                times[i] = min([loc_min[-1,j], times[i]])
    runge_para['start'] = starts[:, np.argmax(times):np.argmax(times)+1]
    print(max(times))
    if max(times) > runge_para["period"][1]*0.99:
        break

## 初期値固定

In [ ]:
#ある程度持続すれば更新
runge_para["period"]=np.array([0,1000])
competitives = 100

#刻み幅
runge_para["step"] = 0.01
skip= 100
#runge_para['start'] = np.load("../laminar_initials/beta0.43229999999999996_nu0.0001433408684811308_step0.01_806.1699999994119period_laminar.npy")

#nu
#runge_para["nu"] =  0.0001433408684811308
#betaの値に従いc_n_2,c_n_3を更新
#beta = 0.4323

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)
model = RungeKutta_acceralated.RungeKutta()
lmodel = Long_laminar()

for _ in range(5):
    #摂動を与えたnuのリスト作成
    nus = np.zeros((competitives),dtype = np.float64)
    nus[0] = runge_para["nu"]*1
    for i in range(competitives-1):
        a=-5
        b=-9
        nus[i+1] = runge_para["nu"] + (10) ** ((b - a) * np.random.rand() + a) * (-1) ** np.random.randint(1,3) 


    #ラミナーから外れた時間
    times = np.zeros((competitives)) + runge_para["period"][1]

    for i in tqdm(range(competitives)):
        runge_para['nu'] = nus[i]
        #インスタンス化&実行
        model.get_arr_latter(**runge_para,latter=1)

        #極大抽出
        loc_max = Long_laminar.loc_max_7(model.arr_latter) #u1が極大値なものを拾ってくる
        loc_min = Long_laminar.loc_min_7(model.arr_latter) #u1が極小値なものを拾ってくる

        #極大値で判定
        for j in range(loc_max.shape[1]):
            if not 0.87 < loc_max[0,j] < 1.1:
                times[i] = loc_max[-1, j]
                break
        #極小値で判定
        for j in range(loc_min.shape[1]):
            if not 0.5 < loc_min[0,j] < 0.64:
                times[i] = min([loc_min[-1,j], times[i]])
    runge_para['nu'] = nus[np.argmax(times)]
    print(max(times))
    if max(times) > runge_para["period"][1]*0.99:
        break
        
if max(times) > 500:
    np.savez_compressed(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{max(times)}period_laminar", start=model.arr_latter[:-1,0:1], laminar=model.arr_latter)

## 初期値・nu ランダム

In [ ]:
%%time
#ある程度持続すれば更新
runge_para["period"]=np.array([0,1000])
competitives = 100

#刻み幅
runge_para["step"] = 0.01
runge_para['start'] = np.load("../laminar_initials/beta0.419_nu0.00017216861171046193_step0.01_6195.000000088802period_laminar.npy")

#nu
runge_para["nu"] = 0.00017216861171046193
beta= 0.419

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

model = RungeKutta_acceralated.RungeKutta()
lmodel = RungeKutta_acceralated.Long_laminar()

for _ in tqdm(range(5)):
    #摂動を与えたnuのリスト作成
    nus = np.zeros((competitives),dtype = np.float64)
    nus[0] = runge_para["nu"]*1
    for i in range(competitives-1):
        a=-4.7
        b=-9
        nus[i+1] = runge_para["nu"] + (10) ** ((b - a) * np.random.rand() + a) * (-1) ** np.random.randint(1,3) 

    #摂動を与えた初期値のリスト作成
    starts = np.zeros((14,competitives),dtype = np.complex128)
    starts[:,0:1] = runge_para["start"].copy()
    for i in range(competitives-1):
        starts[:,i+1:i+2] = lmodel.perturbator(runge_para['start'])

    #ラミナーから外れた時間
    times = np.zeros((competitives)) + runge_para["period"][1]

    for i in range(competitives):
        runge_para['start'] = starts[:,i:i+1].copy()
        runge_para['nu'] = nus[i]
        #インスタンス化&実行
        model.get_arr_latter(**runge_para,latter=1)

        #極大抽出
        loc_max = lmodel.loc_max_7(model.arr_latter, 1) #u1が極大値なものを拾ってくる
        loc_min = lmodel.loc_min_7(model.arr_latter, 1) #u1が極小値なものを拾ってくる

        #極大値で判定
        for j in range(loc_max.shape[1]):
            if not 0.87 < loc_max[0,j] < 1.1:
                times[i] = loc_max[-1, j]
                break
        #極小値で判定
        for j in range(loc_min.shape[1]):
            if not 0.5 < loc_min[0,j] < 0.64:
                times[i] = min([loc_min[-1,j], times[i]])
                break
                
    runge_para['nu'] = nus[np.argmax(times)]
    runge_para['start'] = starts[:, np.argmax(times):np.argmax(times)+1]
    print(max(times))
    if max(times) > runge_para["period"][1]*0.99:
        break

if max(times) > 600:
    np.save(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{max(times)}period_laminar", model.arr_latter[:-1,0:1])

# マルチコア

## nu固定

In [ ]:
%%time
#ある程度持続すれば更新
runge_para["period"]=np.array([0,2000])
competitives = 50

#刻み幅
runge_para["step"] = 0.01
skip= 100
runge_para['start'] = np.load("../laminar_initials/beta0.424_nu0.0001766202254935416_step0.01_1818.579999998491period_laminar.npy")

#nu
runge_para["nu"] = 0.00017662
beta= 0.424
#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)
model = RungeKutta_acceralated.RungeKutta()
lmodel = RungeKutta_acceralated.Long_laminar()

for _ in tqdm(range(4)):
    with concurrent.futures.ProcessPoolExecutor(max_workers = 8) as executor: 
        task1 = executor.submit(func_for_paralell.competition, runge_para, competitives, nu = False)
        task2 = executor.submit(func_for_paralell.competition, runge_para, competitives, nu = False)
        task3 = executor.submit(func_for_paralell.competition, runge_para, competitives, nu = False)
        task4 = executor.submit(func_for_paralell.competition, runge_para, competitives, nu = False)
        # task5 = executor.submit(func_for_paralell.competition, runge_para, competitives, nu = False)
        # task6 = executor.submit(func_for_paralell.competition, runge_para, competitives, nu = False)
        # task7 = executor.submit(func_for_paralell.competition, runge_para, competitives, nu = False)
        times1, starts1, nus1 = task1.result()
        times2, starts2, nus2 = task2.result()
        times3, starts3, nus3 = task3.result()
        times4, starts4, nus4 = task4.result()
        # times5, starts5, nus5 = task5.result()
        # times6, starts6, nus6 = task6.result()
        # times7, starts7, nus7 = task7.result()

    times = np.concatenate([times1, times2, times3, times4])
    starts = np.concatenate((starts1, starts2, starts3, starts4), axis=1)
    #nus = np.concatenate([nus1, nus2, nus3, nus4, nus5, nus6, nus7])
    #runge_para['nu'] = nus[np.argmax(times)]
    runge_para['start'] = starts[:, np.argmax(times):np.argmax(times)+1]
    print(max(times))
    if max(times) > runge_para["period"][1]*0.99:
        break
if max(times) > 1000:
    np.save(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{max(times)}period_laminar", runge_para["start"])

## 初期値固定

In [ ]:
%%time
#ある程度持続すれば更新
runge_para["period"]=np.array([0,2000])
competitives = 100

#刻み幅
runge_para["step"] = 0.01
skip= 100
runge_para['start'] = np.load("../laminar_initials/beta0.4328_nu0.00014159314443410194_step0.01_1000.0period_laminar.npy")

#nu
runge_para["nu"] = 0.00014159314443410194
beta= 0.4328
#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)
model = RungeKutta_acceralated.RungeKutta()
lmodel = RungeKutta_acceralated.Long_laminar()

for _ in tqdm(range(4)):
    with concurrent.futures.ProcessPoolExecutor(max_workers = 8) as executor: 
        task1 = executor.submit(func_for_paralell.competition, runge_para, competitives)
        task2 = executor.submit(func_for_paralell.competition, runge_para, competitives)
        task3 = executor.submit(func_for_paralell.competition, runge_para, competitives)
        task4 = executor.submit(func_for_paralell.competition, runge_para, competitives)
        task5 = executor.submit(func_for_paralell.competition, runge_para, competitives)
        task6 = executor.submit(func_for_paralell.competition, runge_para, competitives)
        task7 = executor.submit(func_for_paralell.competition, runge_para, competitives)

        times1, starts1, nus1 = task1.result()
        times2, starts2, nus2 = task2.result()
        times3, starts3, nus3 = task3.result()
        times4, starts4, nus4 = task4.result()
        times5, starts5, nus5 = task5.result()
        times6, starts6, nus6 = task6.result()
        times7, starts7, nus7 = task7.result()

    times = np.concatenate([times1, times2, times3, times4, times5, times6, times7])
    #starts = np.concatenate((starts1, starts2, starts3, starts4, starts5, starts6, starts7), axis=1)
    nus = np.concatenate([nus1, nus2, nus3, nus4, nus5, nus6, nus7])
    runge_para['nu'] = nus[np.argmax(times)]
    #runge_para['start'] = starts[:, np.argmax(times):np.argmax(times)+1]
    print(max(times))
    if max(times) > runge_para["period"][1]*0.99:
        break
if max(times) > 1000:
    np.save(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{max(times)}period_laminar", runge_para["start"])

## 初期値・nuランダム

In [ ]:
%%time
#縦軸に各変数、横軸時間　×次元数 + flux
runge_para["period"]=np.array([0,10000])

#刻み幅
runge_para["step"] = 0.01
competitives = 100
skip= 100
runge_para['start'] = np.load("../laminar_initials/beta0.419_nu0.00017216861171046193_step0.01_6195.000000088802period_laminar.npy")

#nu
runge_para["nu"] = 0.00017216861171046193
beta= 0.419
#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)
model = RungeKutta_acceralated.RungeKutta()
lmodel = RungeKutta_acceralated.Long_laminar()

for _ in tqdm(range(4)):
    with concurrent.futures.ProcessPoolExecutor(max_workers =8) as executor: 
        task1 = executor.submit(func_for_paralell.competition, runge_para, competitives)
        task2 = executor.submit(func_for_paralell.competition, runge_para, competitives)
        task3 = executor.submit(func_for_paralell.competition, runge_para, competitives)
        task4 = executor.submit(func_for_paralell.competition, runge_para, competitives)
        task5 = executor.submit(func_for_paralell.competition, runge_para, competitives)
        task6 = executor.submit(func_for_paralell.competition, runge_para, competitives)

        times1, starts1, nus1 = task1.result()
        times2, starts2, nus2 = task2.result()
        times3, starts3, nus3 = task3.result()
        times4, starts4, nus4 = task4.result()
        times5, starts5, nus5 = task5.result()
        times6, starts6, nus6 = task6.result()

    times = np.concatenate([times1, times2, times3, times4, times5, times6])
    starts = np.concatenate((starts1, starts2, starts3, starts4, starts5, starts6), axis=1)
    nus = np.concatenate([nus1, nus2, nus3, nus4, nus5, nus6])
    runge_para['nu'] = nus[np.argmax(times)]
    runge_para['start'] = starts[:, np.argmax(times):np.argmax(times)+1]
    print(f"nu={runge_para['nu']},{max(times)}")
    if max(times) > runge_para["period"][1]*0.99:
        break
if max(times) > 1000:
    np.save(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{int(max(times))}period_laminar", runge_para["start"])

In [ ]:
laminar = np.load("ラミナー軌道例.npy")
plt.scatter(np.abs(starts[3,:]),np.abs(starts[4,:]), s=0.1)
plt.plot(abs(laminar[3,200000::10]),abs(laminar[4,200000::10]),color="r",linewidth=0.1,alpha = 0.1)
plt.plot(abs(model.arr_latter[3,:50:skip]),abs(model.arr_latter[4,:50:skip]),lw=0.1)

In [ ]:
#保存
np.savez_compressed(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{max(times)}period_laminar", start=model.arr_latter[:-1,0:1], laminar=model.arr_latter)

In [ ]:
print(runge_para["period"][0])

In [ ]:
#軌道確認
skip = 1
model.get_arr_latter(**runge_para,latter=1)
Nth_shell = 3
plt.figure(figsize=(20,30))
for i in range(Dimention):
    plt.subplot(Dimention+1,1,i+1)
    plt.ylabel(f"u{i+1}")
    #plt.yscale=("log")
    plt.plot(np.real(model.arr_latter[-1,::skip]),abs(model.arr_latter[i,::skip]),lw=1)

plt.subplot(Dimention+1,1,Dimention+1)
plt.ylabel(f"flux of {Nth_shell}th shell")
plt.plot(np.real(model.arr_latter[-1,::]), lmodel.flux(model.arr_latter[:-1,:], Nth_shell, 0.5)[0,:])

In [ ]:
#探索の後に実行すれば分布をプロット
plt.hist(times, bins =100)

# 初期値ランダムnu範囲

In [ ]:
#アニメーション(パラメータ変更)
lmodel = Long_laminar()

runge_para['period']=(0,300)
#runge_para['start'] = np.load("../laminar_initials/beta0.4351100000000018_nu0.00013515513078827336_step0.01_928.7999999993003period_laminar.npy")
runge_para['step'] = 0.005

param_step = 100 #パラメータ何回刻む

starts = np.zeros((14,param_step),dtype = np.complex128)
for i in range(param_step):
    starts[:,i:i+1] = lmodel.perturbator(runge_para['start']) #初期値生成

nu_begin = 0.00008 #パラメータ開始
nu_end = 0.0001#パラメータ終わり

beta = 0.5
#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

latter = 1 #後半何分の1をプロットするか
skip = 10 #各パラメータでプロットする際、何刻み分飛ばしでプロットするか(大きければカクカク。メモリ対策)
#参考までにdt0.00025,9000-10000時間では1000skipぐらいなら気にならない。10000だと酷い。

nu_param = np.linspace(nu_begin,nu_end,param_step) 
#nu_param=[math.pow(10,i) for i in nu_param] #log10の時は実行
step_number = int((runge_para["period"][1] - runge_para["period"][0]) // runge_para["step"]) + 1
#kaishi = step_number // latter

laminar = np.load("ラミナー軌道例.npy")
 

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()

#プロットの設定
fig, ax = plt.subplots(dpi=100)
ims = []
ax.set_xlabel("|u4|")
ax.set_ylabel("|u5|")

for i in tqdm(range(param_step)):
    #nu更新
    runge_para["nu"] = nu_param[i]
    
    #初期値設定
    runge_para['start'] = starts[:, i:i+1]
    
    
    model.get_arr_latter(**runge_para,latter=latter)

    # #次のパラメータの初期値を今回の最後の値に変更
    # runge_para["start"] = model.arr[:-1,-1:]
    
    text = ax.text(0.5,1.05,(f"beta={beta} \n nu={nu_param[i]}"), size = 15, color = "green", ha = "center", transform=ax.transAxes)
    
    im1 = ax.plot(abs(model.arr_latter[1,::skip]),abs(model.arr_latter[2,::skip]),color="b",linewidth=0.1)
    im2 = ax.plot(abs(laminar[1,200000::10]),abs(laminar[2,200000::10]),color="r",linewidth=0.1,alpha = 0.1)
    ims.append(im1 + im2 +[text])

anim = animation.ArtistAnimation(fig, ims)
#anim.save(f"../animation_attracter_per_parameter/nu({nu_param[0]},{nu_param[-1]})_beta({beta_param[0]},{beta_param[-1]})_{runge_para['step']}_{runge_para['period']}_{param_step}cut_{Dimention}dim_latter{round(1/latter)}.mp4")
HTML(anim.to_jshtml())